# **Crypto forecasting tutorial / 暗号通貨予測のチュートリアル**

# G-Research Crypto forecasting competition / G-Research 暗号通貨予測コンペティション

In the [G-Research Crypto forecasting competition](link), participants have the challenge to predict  price returns across a bundle of major cryptocurrencies. To facilitate your participation, we have created this tutorial notebook covering some relevant concepts for the crypto forecasting challenge.

G-Research 暗号通貨予測コンペティションでは、参加者は主要な暗号通貨群の価格リターンを予測するという課題が与えれれます。参加しやすいように、このチュートリアルノートでは、暗号通貨予測コンペティションに関連するコンセプトを説明しています。

The notebook presents an introduction to crypto forecasting, describing the structure and elements of the dataset, some relevant statistical properties, as well as building a couple of ML baseline models and providing an example code submission.

このノートブックでは、暗号通貨予測について紹介し、データセットの構造と要素、関連する統計的特性を説明するとともに、いくつかの機械学習ベースラインモデルを構築し、コード投稿の例を示しています。

## The Cryptocurrency Market / 暗号通貨市場について

First, a quick introduction to the crypto world. Cryptocurrencies have become an extremely popular and volatile market, delivering massive returns (as well as losses) to investors. Thousands of cryptocurrencies have been created with a few major ones that many of you will have heard of including Bitcoin (BTC), Ether (ETH) or Dogecoin (DOGE).

まず、暗号通貨の世界について簡単に紹介します。暗号通貨は非常に人気が高く、変動の激しい市場となっており、投資家に多額のリターン（損失も含む）をもたらしています。何千もの暗号通貨が作られていますが、主なものとしては、ビットコイン（BTC）、イーサ（ETH）、ドージコイン（DOGE）などがあり、多くの人が聞いたことがあるでしょう。

Cryptocurrencies are traded extensively across crypto-exchanges, with an average volume of $41 billion traded daily over the last year, according to CryptoCompare (as of 25th July 2021). 

暗号通貨は暗号通貨取引所で広く取引されており、CryptoCompareによると、昨年1年間で毎日平均410億ドルの取引が行われています（2021年7月25日現在）。

Changes in prices between different cryptocurrencies are highly interconnected. For example, Bitcoin has historically been a major driver of price changes across cryptocurrencies but other coins also impact the market.  

異なる暗号通貨間の価格の変化は、高度に相互に関連しています。例えば、ビットコインは歴史的に暗号通貨間の価格変動の主な要因となってきましたが、他のコインも市場に影響を与えます。


## Forecasting returns / リターンの予測

A fundamental task in financial modeling is predicting how prices will behave in the near future. Using the time-series of historical prices as training data, we want to predict if prices will go up or down, and by how much, namely the asset *returns*.

金融モデリングの基本的な課題は、近い将来の価格がどのように推移するかを予測することです。過去の価格の時系列を学習データとして、価格が上がるのか下がるのか、どれくらい上がるのかを予測したい、つまり資産のリターンを予測したいのです。

In this competition, Kagglers are challenged to build machine learning models to predict the returns of 14 popular cryptocurrencies, in the time scale of minutes to hours. You will have access to millions of rows of minute-by-minute cryptocurrency trading data, with which you'll design your forecasting models for all 14 assets simultaneously. Your predictions will be evaluated by how much they correlate with real market data collected during the three-month evaluation period after the competition has closed. 

本コンペティションでは、人気のある14種類の暗号通貨のリターンを予測する機械学習モデルを、数分から数時間の時間スケールで構築することがKagglersの課題となっています。数百万行の分刻みの暗号通貨取引データにアクセスし、それをもとに14の資産すべてを同時に対象とした予測モデルを設計することになります。あなたの予測は、コンテスト終了後の3ヶ月間の評価期間中に収集された実際の市場データとどれだけ相関しているかによって評価されます。

Cryptocurrency returns prediction remains an open and extremely challenging forecasting task. This is a fascinating problem domain for the ML community given the extreme volatility of the assets, the non-stationary nature of the data, the market and meme manipulation, the correlation between assets and the very fast changing market conditions. We hope you find it as fascinating as we do! 

暗号通貨のリターン予測は、依然として未解決で、非常に困難な予測課題です。これは、資産の極端な変動性、データの非定常性、市場や操作、資産間の相関性、非常に速い市場状況の変化を考えると、機械学習コミュニティにとって魅力的な問題領域です。私たちと同じように、皆さんにも魅力的な問題を見つけていただきたいと思います。

# Dataset description / データセットの説明

Now, let's dive into the data! We start by loading the competition's dataset and inspecting its basic properties. 

それでは、データを見てみましょう。まずは、コンペティションのデータセットを読み込んで、その基本的なプロパティを確認します。

## Load the training set / トレーニングセットの読み込み

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
data_folder = "../input/g-research-crypto-forecasting/"
!ls $data_folder

In [ ]:
crypto_df = pd.read_csv(data_folder + 'train.csv')

In [ ]:
crypto_df.head(10)

We can see that each row of the data set has the trading data for an asset, at a given minute timestamp, described in detail below. 

データセットの各行には、特定の分のタイムスタンプにおける資産の取引データがあることがわかります。

## Data features / データの特徴

We can see the different features included in the dataset. Specifically, the features included per asset are the following:

データセットに含まれる様々な特徴を見ることができます。具体的には、アセットごとに含まれている特徴は以下の通りです。

*   **timestamp**: All timestamps are returned as second Unix timestamps (the number of seconds elapsed since 1970-01-01 00:00:00.000 UTC). Timestamps in this dataset are multiple of 60, indicating minute-by-minute data. / すべてのタイムスタンプは、秒単位のUnixタイムスタンプ（1970-01-01 00:00:00.000 UTCからの経過秒数）で返されます。このデータセットのタイムスタンプは60の倍数で、1分ごとのデータを示しています。
*   **Asset_ID**: The asset ID corresponding to one of the crytocurrencies (e.g. `Asset_ID = 1` for Bitcoin). The mapping from `Asset_ID` to crypto asset is contained in `asset_details.csv`. / いずれかの暗号通貨に対応するアセットID（例：Bitcoinの場合は`Asset_ID = 1`）。`Asset_ID`から暗号資産へのマッピングは`asset_details.csv`に含まれる。
*   **Count**: Total number of trades in the time interval (last minute). / 時間間隔（最後の1分）での取引の総数。
*   **Open**:	Opening price of the time interval (in USD). / 時間間隔内の始値（単位：米ドル）。
*   **High**:	Highest price reached during time interval (in USD). / 時間間隔内の高値（単位：米ドル）。
*   **Low**: Lowest price reached during time interval (in USD). / 時間間隔内の安値（単位：米ドル）。
*   **Close**:	Closing price of the time interval (in USD). / 時間間隔内の終値（米ドル）。
*   **Volume**:	Quantity of asset bought or sold, displayed in base currency USD. / 買ったまたは売った資産の量で、基本通貨の米ドルで表示されます。
*   **VWAP**: The average price of the asset over the time interval, weighted by volume. VWAP is an aggregated form of trade data. / 取引時間中の平均価格を取引量で加重平均したもの。VWAPは取引データを集計したものです。
*   **Target**: Residual log-returns for the asset over a 15 minute horizon. / 15分間の暗号資産の残余対数リターン。

The first two columns define the time and asset indexes for this data row. The 6 middle columns are feature columns with the trading data for this asset and minute in time. The last column is prediction target, which we will get to later in more detail.

最初の2列は、このデータ行の時間と資産のインデックスを定義しています。中央の6列は、この資産と時間の分の取引データを含む特徴的な列です。最後の列は予測ターゲットですが、これについては後で詳しく説明します。

We also view the asset information, including the list of all assets, the `Asset_ID` to asset mapping, and the weight of each asset used to weigh their relative importance in the evaluation metric.

また、すべてのアセットのリスト、アセット_IDとアセットのマッピング、評価指標における相対的な重要性を評価するために使用される各アセットの重みなど、アセット情報も表示されます。

In [ ]:
asset_details = pd.read_csv(data_folder + 'asset_details.csv')
asset_details

## Candlestick charts / ローソク足チャート

The trading data format is an aggregated form of market data including for Open, High, Low and Close. We can visualize this data through the commonly used candlestick bar chart, which allows traders to perform technical analysis on intraday values. The bar's body length represents the price range between the open and close of that day's trading. When the bar is red, it means the close was lower than the open, and green otherwise. These are also referred to as bullish and bearish candlesticks. The wicks above and below the bars show the high and low prices of that interval's trading.

取引データのフォーマットは、Open、High、Low、Closeを含む市場データを集約したものです。このデータは、一般的に使用されているローソク足のバーチャートで視覚化することができ、トレーダーは日中の値でテクニカル分析を行うことができます。バーの長さは、その日の取引の始値と終値の間の価格帯を表しています。バーの色が赤の場合は、終値が始値よりも低かったことを意味し、そうでない場合は緑となる。これを強気のローソク足、弱気のローソク足ともいう。バーの上下にある芯は、その区間の取引の高値と安値を示しています。

We can visualize a slice of the Bitcoin prices using the `plotly` library. The bottom part of the plot shows a rangeslider, which you can use to zoom in the plot.

`plotly`ライブラリを使用して、Bitcoin価格のスライスを視覚化することができます。プロットの下部にはレンジスライダーが表示されており、これを使ってプロットをズームインすることができます。

In [ ]:
btc = crypto_df[crypto_df["Asset_ID"]==1].set_index("timestamp") # Asset_ID = 1 for Bitcoin
btc_mini = btc.iloc[-200:] # Select recent data rows

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=btc_mini.index, open=btc_mini['Open'], high=btc_mini['High'], low=btc_mini['Low'], close=btc_mini['Close'])])
fig.show()

# Preprocessing / 前処理

## Dealing with missing data / 欠損データの処理


Let us inspect the data for another important asset, Ethereum.

もう一つの重要な暗号資産であるEthereumのデータを調べてみましょう。

In [ ]:
eth = crypto_df[crypto_df["Asset_ID"]==6].set_index("timestamp") # Asset_ID = 6 for Ethereum
eth.info(show_counts =True)

We can see the number of rows in the training set, and that there are missing values for the targets columns, which we will address later. Let's confirm that:

トレーニングセットの行数を確認し、ターゲットの列に欠損値があることがわかりますが、これについては後ほど説明します。確認してみましょう。

In [ ]:
eth.isna().sum()

Let's check the time range for Bitcoin and Ethereum data, using the coversion from timestamp to `datetime`.

timestampから`datetime`へ変換して、BitcoinとEthereumのデータの時間範囲を確認してみましょう。

In [ ]:
btc.head()

In [ ]:
beg_btc = btc.index[0].astype('datetime64[s]')
end_btc = btc.index[-1].astype('datetime64[s]')
beg_eth = eth.index[0].astype('datetime64[s]')
end_eth = eth.index[-1].astype('datetime64[s]')

print('BTC data goes from ', beg_btc, 'to ', end_btc)
print('Ethereum data goes from ', beg_eth, 'to ', end_eth)

Missing asset data, for a given minute, is not represented by NaN's, but instead by the absence of those rows. We can check the timestamp difference between consecutive rows to see if there is missing data.

暗号資産データの欠落は、NaNではなく、それらの行がないことで表現されます。欠損データがあるかどうかは、連続した行のタイムスタンプの差を確認することができます。

In [ ]:
(eth.index[1:]-eth.index[:-1]).value_counts().head()

Notice that there are many gaps in the data. To work with most time series models, we should preprocess our data into a format without time gaps. To fill the gaps, we can use the `.reindex()` method for forward filling, filling gaps with the previous valid value. 

データには多くのギャップがあることに注意してください。多くの時系列モデルを扱うためには、データを時間のギャップのないフォーマットに前処理する必要があります。ギャップを埋めるためには、`.reindex()`メソッドを使って、前回の有効な値でギャップを埋めるフォワードフィリングを行います。

In [ ]:
eth = eth.reindex(range(eth.index[0],eth.index[-1]+60,60),method='pad')

And check that are no time gaps now.

時間的なギャップがないことを確認します。

In [ ]:
(eth.index[1:]-eth.index[:-1]).value_counts().head()

## Data visualisation / データの可視化

We  will start by visualising the Close prices for the two assets we have selected.

選択した2つの暗号資産の終値を可視化してみます。

In [ ]:
import matplotlib.pyplot as plt

# plot vwap time series for both chosen assets
f = plt.figure(figsize=(15,4))

# fill missing values for BTC
btc = btc.reindex(range(btc.index[0],btc.index[-1]+60,60),method='pad')

ax = f.add_subplot(121)
plt.plot(btc['Close'], label='BTC')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Bitcoin')

ax2 = f.add_subplot(122)
ax2.plot(eth['Close'], color='red', label='ETH')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Ethereum')

plt.tight_layout()
plt.show()

The assets have quite different history, but we could check if they correlate in recent times.

両暗号資産の歴史は全く異なりますが、最近では相関関係があることを確認できました。

In [ ]:
import time

# auxiliary function, from datetime to timestamp
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))

# create intervals
btc_mini_2021 = btc.loc[totimestamp('01/06/2021'):totimestamp('01/07/2021')]
eth_mini_2021 = eth.loc[totimestamp('01/06/2021'):totimestamp('01/07/2021')]

In [ ]:
# plot time series for both chosen assets
f = plt.figure(figsize=(7,8))

ax = f.add_subplot(211)
plt.plot(btc_mini_2021['Close'], label='btc')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Bitcoin Close')

ax2 = f.add_subplot(212)
ax2.plot(eth_mini_2021['Close'], color='red', label='eth')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Ethereum Close')

plt.tight_layout()
plt.show()

On shorter intervals we can visually see some potential correlation between both assets, with some simultaneous ups and downs. A better format for analyzing such movements is by calculating asset returns. 

もっと短い間隔で見ると、両暗号資産の間には潜在的な相関関係があり、同時にいくつかの上昇と下降があることが視覚的にわかります。このような動きを分析するには、資産のリターンを計算するのが良いでしょう。

## Log returns / 対数収益率

In order to analyze price changes for an asset we can deal with the price difference. However, different assets exhibit different price scales, so that the their returns are not readily comparable. We can solve this problem by computing the percentage change in price instead, also known as the return. This return coincides with the percentage change in our invested capital.

ある資産の価格変動を分析するためには、価格差を扱うことができます。しかし、異なる資産は異なる価格スケールを示すため、そのリターンは容易に比較できません。この問題を解決するには、価格の変化率（リターン）を計算する必要があります。このリターンは、投資資本の変化率と一致します。

Returns are widely used in finance, however log returns are preferred for mathematical modelling of time series, as they are additive across time. Also, while regular returns cannot go below -100%, log returns are not bounded.

リターンは金融の分野で広く使われていますが、時系列の数学的モデリングには、時間を超えて加法的に変化する対数収益率が適しています。また、通常のリターンは-100%以下になることはありませんが、対数収益率には制限がありません。

To compute the log return, we can simply take the logarithm of the ratio between two consecutive prices. The first row will have an empty return as the previous value is unknown, therefore the empty return data point will be dropped.

対数収益率を計算するには、単純に連続する2つの価格間の比率の対数を取ることができます。最初の行は、前の値が不明であるため、空のリターンがあり、したがって、空のリターンのデータポイントは削除されます。

In [ ]:
# define function to compute log returns
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

We can visualize the log return for our two assets. See how the signal now looks more like white noise, with less drift than the time series for prices.

2つの暗号資産の対数収益率を可視化します。シグナルがホワイトノイズのようになり、価格の時系列に比べてドリフトが少なくなっているのがわかります。

In [ ]:
import scipy.stats as stats

lret_btc = log_return(btc_mini_2021.Close)[1:]
lret_eth = log_return(eth_mini_2021.Close)[1:]
lret_btc.rename('lret_btc', inplace=True)
lret_eth.rename('lret_eth', inplace=True)

plt.figure(figsize=(8,4))
plt.plot(lret_btc);
plt.plot(lret_eth);
plt.show()

## Correlation between assets / 資産間の相関関係

We hypothesized before that crypto asset returns may exhibit some correlation. Let's check this in more detail now.

私たちは以前、暗号資産のリターンには何らかの相関関係があるのではないかという仮説を立てました。今度はこれをより詳細に確認してみましょう。

We can check how the correlation between Bitcoin and Ethereum change over time for the 2021 period we selected. 

私たちが選んだ2021年の期間において、ビットコインとイーサリアムの相関関係が時系列でどのように変化するかを確認します。

In [ ]:
# join two asset in single DataFrame

lret_btc_long = log_return(btc.Close)[1:]
lret_eth_long = log_return(eth.Close)[1:]
lret_btc_long.rename('lret_btc', inplace=True)
lret_eth_long.rename('lret_eth', inplace=True)
two_assets = pd.concat([lret_btc_long, lret_eth_long], axis=1)

# group consecutive rows and use .corr() for correlation between columns
corr_time = two_assets.groupby(two_assets.index//(10000*60)).corr().loc[:,"lret_btc"].loc[:,"lret_eth"]

corr_time.plot();
plt.xticks([])
plt.ylabel("Correlation")
plt.title("Correlation between BTC and ETH over time");

Note the high but variable correlation between the assets. Here we can see that there is some changing dynamics over time, and this would be critical for this time series challenge, that is, how to perform forecasts in a highly non-stationary environment.

資産間の相関性は高いものの、変動していることに注意してください。ここでは、時間とともに変化するダイナミクスがあることがわかります。このことは、今回の時系列の課題、つまり高度に非定常な環境でどのように予測を実行するかという点で重要になります。

A stationary behaviour of a system or a process is characterized by non-changing statistical properties over time such as the mean, variance and autocorrelation. On the other hand, a non-stationary behaviour is characterized by a continuous change of statistical properties over time. Stationarity is important because many useful analytical tools and statistical tests and models rely on it.

システムやプロセスの定常的な挙動は、平均、分散、自己相関などの時間的に変化しない統計的特性によって特徴付けられます。一方、非定常挙動は、統計的特性が時間とともに連続的に変化することを特徴とします。定常性は、多くの有用な分析ツールや統計的なテストやモデルがそれに依存しているため、重要です。

We can also check the correlation between all assets visualizing the correlation matrix. Note how some assets have much higher pairwise correlation than others.

また、相関行列を可視化することで、すべての資産間の相関を確認することができます。いくつかの資産が他の資産よりもはるかに高いペアワイズ相関を持っていることに注意してください。

In [ ]:
# create dataframe with returns for all assets
all_assets_2021 = pd.DataFrame([])
for asset_id, asset_name in zip(asset_details.Asset_ID, asset_details.Asset_Name):
  asset = crypto_df[crypto_df["Asset_ID"]==asset_id].set_index("timestamp")
  asset = asset.loc[totimestamp('01/01/2021'):totimestamp('01/05/2021')]
  asset = asset.reindex(range(asset.index[0],asset.index[-1]+60,60),method='pad')
  lret = log_return(asset.Close.fillna(0))[1:]
  all_assets_2021 = all_assets_2021.join(lret, rsuffix=asset_name, how="outer")

In [ ]:
plt.imshow(all_assets_2021.corr());
plt.yticks(asset_details.Asset_ID.values, asset_details.Asset_Name.values);
plt.xticks(asset_details.Asset_ID.values, asset_details.Asset_Name.values, rotation='vertical');
plt.colorbar();

We encourage participants to perform additional statistical analyses to have a stronger grasp on the dataset, including autocorrelation, time-series decomposition and stationarity tests.

皆さんには、データセットをより正確に把握するために、自己相関、時系列分解、定常性の検定など、追加の統計解析を行っていただきたいと思います。

# Building your prediction model / 予測モデルの構築

## Prediction targets and evaluation / 予測対象と評価

This forecasting competition aims to predict returns in the near future for prices $P^a$, for each asset $a$. For each row in the dataset, we include the target for prediction, `Target`. `Target` is derived from log returns ($R^a$) over 15 minutes.

この予測コンペティションは、各資産$a$について、価格$P^a$に対する近未来のリターンを予測することを目的としています。データセットの各行には、予測のターゲットである`Target`が含まれる。`Target`は15分間の対数収益率($R^a$)から得られる。

$$R^a(t) = log (P^a(t+16)\ /\ P^a(t+1))$$

Crypto asset returns are highly correlated, following to a large extend the overall crypto market. As we want to test your ability to predict returns for individual assets, we perform a linear residualization, removing the market signal from individual asset returns when creating the two targets. In more detail, if $M(t)$ is the weighted average market returns, the target is:

暗号資産のリターンは非常に相関性が高く、暗号市場全体を大きくフォローしています。個々の資産のリターンを予測する能力をテストしたいため、2つのターゲットを作成する際には、個々の資産のリターンから市場のシグナルを除去し、線形残差を行います。より詳細には、$M(t)$が加重平均市場リターンである場合、ターゲットは

$$M(t) = \frac{\sum_a w^a R^a(t)}{\sum_a w^a}  \\
\beta^a = \frac{\langle M \cdot R^a \rangle}{\langle M^2 \rangle} \\
\text{Target}^a(t) = R^a(t) - \beta^a M(t)$$

where the bracket $\langle .\rangle$ represent the rolling average over time (3750 minute windows). 

ここで、括弧内の$\langle .\rangle$は、時間（3750分窓）でのローリング平均を表しています。

Some rows have null values for targets due to missing values in future prices. Rows with nulls in the test set ground truth are ignored for scoring purposes.

一部の行では、将来の価格の値が欠落しているため、ターゲットの値が欠損値になっています。テストセットのに欠損値がある行は、スコアリングのために無視されます。

In the competition, your predictions will be evaluated on a weighted version of the Pearson correlation coefficient, with weights given by the `Weight` column in the Asset Details file.

コンペティションでは、あなたの予測は、ピアソン相関係数の加重バージョンで評価されます。加重は、資産詳細ファイルの Weight 列で与えられます。

In this tutorial, we will simplify things and use correlation (without weights) for evaluation, and consider only two assets, BTC and ETH.

このチュートリアルでは、物事を単純化して、相関関係（重みなし）を評価に使用し、BTC と ETH の 2 つの資産のみを検討します。

## Feature design / 特徴量設計

We first design a few relevant features to input to our model.

まず、モデルに入力するための関連する特徴量をいくつか設計します。

In [ ]:
# Select some input features from the trading data: 
# 5 min log return, abs(5 min log return), upper shadow, and lower shadow.
upper_shadow = lambda asset: asset.High - np.maximum(asset.Close,asset.Open)
lower_shadow = lambda asset: np.minimum(asset.Close,asset.Open)- asset.Low

X_btc = pd.concat([log_return(btc.VWAP,periods=5), log_return(btc.VWAP,periods=1).abs(), 
               upper_shadow(btc), lower_shadow(btc)], axis=1)
y_btc = btc.Target

X_eth = pd.concat([log_return(eth.VWAP,periods=5), log_return(eth.VWAP,periods=1).abs(), 
               upper_shadow(eth), lower_shadow(eth)], axis=1)
y_eth = eth.Target

## Preparing the data for building predictive models / 予測モデル構築のためのデータの準備

As we will train linear regression parameters, we need to separate training and test sets. To do so, we will compute X and y and split this data into train and test splits. Note that the test split represents a later part of the data, as it is commonly done in time series. 

線形回帰パラメータをトレーニングするので、トレーニングセットとテストセットを分ける必要があります。そのために、XとYを計算し、このデータを訓練用とテスト用に分割します。時系列でよく行われるように、テスト分割はデータの後の部分を使用していることに注意してください。

In [ ]:
# select training and test periods
train_window = [totimestamp("01/05/2021"), totimestamp("30/05/2021")]
test_window = [totimestamp("01/06/2021"), totimestamp("30/06/2021")]

# divide data into train and test, compute X and y
# we aim to build simple regression models using a window_size of 1
X_btc_train = X_btc.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()  # filling NaN's with zeros
y_btc_train = y_btc.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()  

X_btc_test = X_btc.loc[test_window[0]:test_window[1]].fillna(0).to_numpy() 
y_btc_test = y_btc.loc[test_window[0]:test_window[1]].fillna(0).to_numpy() 

X_eth_train = X_eth.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()  
y_eth_train = y_eth.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()  

X_eth_test = X_eth.loc[test_window[0]:test_window[1]].fillna(0).to_numpy() 
y_eth_test = y_eth.loc[test_window[0]:test_window[1]].fillna(0).to_numpy() 

We now standardize the input data. Standardization is the process of putting different variables on the same scale. In regression analysis, it is often crucial to standardize your independent variables or you may risk obtaining misleading results.

次に、入力データの標準化を行います。標準化とは、異なる変数を同じ尺度にするプロセスです。回帰分析では、独立変数を標準化することが重要な場合が多く、そうしないと誤解を招くような結果を得る危険性があります。

In [ ]:
from sklearn.preprocessing import StandardScaler
# simple preprocessing of the data 
scaler = StandardScaler()

X_btc_train_scaled = scaler.fit_transform(X_btc_train)
X_btc_test_scaled = scaler.transform(X_btc_test)

X_eth_train_scaled = scaler.fit_transform(X_eth_train)
X_eth_test_scaled = scaler.transform(X_eth_test)

## Baseline model: Linear Regression / ベースラインモデル: 線形回帰

We will try a simple Linear Regression model on the features we designed. Note that Linear Regression is not commonly used in time series analysis, specially with only one time step! 

ここでは、我々が設計した特徴量について、単純な線形回帰モデルを試してみます。線形回帰は、時系列分析では一般的に使用されないことに注意してください。

We compare two Linear Regression baselines, one that considers each asset independently and one multiple inputs that models all assets together. 

2つの線形回帰ベースラインを比較します。1つは各資産を独立して考慮するもの、もう1つはすべての資産を一緒にモデル化する複数の入力です。

In [ ]:
from sklearn.linear_model import LinearRegression

# implement basic ML baseline (one per asset)
lr = LinearRegression()
lr.fit(X_btc_train_scaled,y_btc_train)
y_pred_lr_btc = lr.predict(X_btc_test_scaled)

lr.fit(X_eth_train_scaled,y_eth_train)
y_pred_lr_eth = lr.predict(X_eth_test_scaled)

In [ ]:
# implement more complex baseline (multiple output regression model)
from sklearn.multioutput import MultiOutputRegressor

# we concatenate X and y for both assets
X_both_train = np.concatenate((X_btc_train_scaled, X_eth_train_scaled), axis=1)
X_both_test = np.concatenate((X_btc_test_scaled, X_eth_test_scaled), axis=1)
y_both_train = np.column_stack((y_btc_train, y_eth_train))
y_both_test = np.column_stack((y_btc_test, y_eth_test))

# define the direct multioutput model and fit it
mlr = MultiOutputRegressor(LinearRegression())
lr.fit(X_both_train,y_both_train)
y_pred_lr_both = lr.predict(X_both_test)

## Evaluate baselines / ベースラインの評価

The competition performance metric is weighted correlation. However, for now we will use simple correlation to evaluate the two baseline models built.

コンペティションの評価指標は加重相関です。しかし、ここでは単純相関を用いて、構築した2つのベースラインモデルを評価します。

In [ ]:
print('Test score for LR baseline: BTC', f"{np.corrcoef(y_pred_lr_btc, y_btc_test)[0,1]:.2f}", 
                                ', ETH', f"{np.corrcoef(y_pred_lr_eth, y_eth_test)[0,1]:.2f}")
print('Test score for multiple output LR baseline: BTC', f"{np.corrcoef(y_pred_lr_both[:,0], y_btc_test)[0,1]:.2f}", 
                                                ', ETH', f"{np.corrcoef(y_pred_lr_both[:,1], y_eth_test)[0,1]:.2f}")

We can see that, for the training and test periods selected, the multiple asset LR model performs better than simply modelling each asset separately. Note that because the data is highly non-stationary, these results might vary a lot for different periods.


選択したトレーニング期間およびテスト期間において、複数資産線形回帰モデルは、単に各資産を個別にモデル化するよりも優れたパフォーマンスを示していることがわかります。なお、データは非定常性が高いため、この結果は期間によって大きく異なる可能性があります。

## Submission / 提出方法

Note that this is a Code Competition, in which you must submit your notebook to be run against the hidden private data. Your notebook should use the provided python time-series API, which ensures that models do not peek forward in time. To use the API, follow the instructions and template in [Code Competition Detailed API instructions](https://www.kaggle.com/sohier/detailed-api-introduction) and [Basic Submission Template](https://www.kaggle.com/sohier/basic-submission-template).

これはコードコンペティションであり、隠されたプライベートデータに対して実行されるノートブックを提出する必要があります。ノートブックは、提供されているpython時系列APIを使用する必要があります。これにより、モデルが時間的に先に進むことがないようになっています。APIを使用するには、[Code Competition Detailed API instructions](https://www.kaggle.com/sohier/detailed-api-introduction)と[Basic Submission Template](https://www.kaggle.com/sohier/basic-submission-template)の指示とテンプレートに従ってください。